In [1]:
# %load C:\Users\Walter\Desktop\tools\template\plot_template.py

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy import stats
import metpy.calc as mpcalc         # for wet-bulb temperature calculation
from metpy.units import units       # for wet-bulb temperature calculation

from sklearn.linear_model import LinearRegression

plt.rc('font', family='serif')
plt.rc('axes', labelsize=16)
plt.rc('xtick', labelsize=14, color='grey')
plt.rc('ytick', labelsize=14, color='grey')
plt.rc('legend', fontsize=16, loc='lower left')
plt.rc('savefig', dpi=330, bbox='tight')
%matplotlib inline


## Define utility functions
#### Read data function

In [169]:
def readData(filename):   # resample 15min
    df = pd.read_csv('data/'+filename+'.csv', index_col=0)
    if 'Excel Time' in df.columns:
        df.drop('Excel Time',axis=1,inplace=True)
    df.index = pd.to_datetime(df.index)
    df = df.resample('15min').mean()
    return df

#### Data quality check

In [5]:
def power_freq_cubed_check(data):
    flag_on = (data['Power, kW']>0)&(data['Actual Hz']>0)
    data_on = data[flag_on]
    x = data_on['Power, kW']/(data_on['Actual Hz']**3)
    x_outlierFree = x[(np.abs(stats.zscore(x)) < 0.5)]
    outlier_ratio = 1-len(x_outlierFree)/len(x)
    std = x_outlierFree.std()/x_outlierFree.mean()
    sns.distplot(x_outlierFree/x_outlierFree.mean(),bins=30)
    print(' removed outliers: {} \n std/mean: {}'.format(outlier_ratio,std))

## Calculate the wet-bulb temperature

In [94]:
weather = readData('weather').dropna()

wb_temp_list = []
for i in range(weather.shape[0]):
    wb_temp = mpcalc.wet_bulb_temperature(1000* units.mbar,  # pressure is needed for wet-bulb temp calculation
                                          weather.iloc[0,2].tolist()* units.degF, 
                                          weather.iloc[0,1].tolist()* units.degF)
    wb_temp_list.append(float(str(round(wb_temp,2)).split(" ")[0]))

weather['wb_temp degC'] = wb_temp_list
weather.to_csv('data/weather.csv')

weather.head()

C:\Users\Walter\Anaconda3\lib\site-packages\dateutil\parser\_parser.py:1206: UnknownTimezoneWarning: tzname PDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)
C:\Users\Walter\Anaconda3\lib\site-packages\dateutil\parser\_parser.py:1206: UnknownTimezoneWarning: tzname PST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


,dew point,humidity,temperature
Date,,,
2018-07-01 00:00:00,60.0,53.0,79.0
2018-07-01 00:15:00,60.0,53.0,79.0
2018-07-01 00:30:00,60.0,60.0,75.0
2018-07-01 00:45:00,59.0,56.0,76.0
2018-07-01 01:00:00,59.0,57.0,75.0


## Prepare the data

In [243]:
def prepare_data(CT_name, threshold=0.3):
    '''
    input 
    CT_name: string
    threshold: power_to_fan_speed_ratio should be close to 1, using this criteria to filter out bad data
               After the filtering, the power_to_fan_speed_ratio should be in the range [1-threshold,1+threshold] 
    '''
    CT1 = readData(CT_name)
    weather = readData('weather')
    cond_fr = readData('CT_water flow')
    
    # clean flow rate data
    def clean_flowRate(data):
        for field in data.columns:
            data.loc[data[field]<200,field]=0
        return data

    cond_fr_clean = clean_flowRate(cond_fr)
    
    data = pd.concat([CT1, weather, cond_fr_clean], axis=1).dropna()
    # extract data when CT is on, which is considered as the fan speed is above 10 HZ
    data_on = data[data['Actual Hz']>10]
    
    power_fanSpeed_cub_ratio = data_on['Power, kW']/(data_on['Actual Hz']**3)

    flag_bad_data = (np.abs(stats.zscore(power_fanSpeed_cub_ratio)) > threshold)
    print('filtering out {} data points according to the power_to_fanSpeed ratio criterion'.format(sum(flag_bad_data)/data_on.shape[0]))
    data_on = data_on[~ flag_bad_data]                                                              # element wise not should use ~ rather than not
        
    data_clean = pd.DataFrame()
    data_clean['water_outlet'] = (data_on['CW Lvg Temp']-32)*5/9
    data_clean['water_inlet'] = (data_on['CW Etg Temp']-32)*5/9
    data_clean['Twb'] = data_on['wb_temp degC']
    
    # FRwater
    ## Assumption: the condenser water are distributed equally between different CT
    cond_totalFlowRate = data_on.loc[:,['CH-1 CW Flow','CH-2 CW Flow','CH-3 CW Flow','CH-4 CHW Flow','CH-5 CHW Flow']].sum(axis=1)
    norFlowRate = cond_totalFlowRate.quantile(.99)
    data_clean['FRwater'] = cond_totalFlowRate.div(norFlowRate)
    
    data_clean['FRair'] = data_on['Actual Hz']/60  ## assumption: proportion to the fan speed
    
    return data_clean

## Regression model
Reference: Energyplus Engineering Reference Page 990

In [204]:
def CT_coolTools(data):
    # model reference: E+ Engineering Reference Page 990
    approach = data['water_outlet']-data['Twb']
    Tr = data['water_inlet']-data['water_outlet']
    df_op = pd.DataFrame()
    df_op['FRair'] = data['FRair']
    df_op['FRair_sq'] = data['FRair']**2
    df_op['FRair_tri'] = data['FRair']**3
    df_op['FRwater'] = data['FRwater']
    df_op['FRair*FRwater'] = data['FRair']*data['FRwater']
    df_op['FRair_sq*FRwater'] = (data['FRair']**2)*data['FRwater']
    df_op['FRwater_sq'] = data['FRwater']**2
    df_op['FRair*FRwater_sq'] = data['FRair']*(data['FRwater']**2)
    df_op['FRwater_tri'] = data['FRwater']**3
    df_op['Twb'] = data['Twb']
    df_op['FRair*Twb'] = data['FRair']*data['Twb']
    df_op['FRair_sq*Twb'] = (data['FRair']**2)*data['Twb']
    df_op['FRwater*Twb'] = data['FRwater']*data['Twb']
    df_op['FRair*FRwater*Twb'] = data['FRair']*data['FRwater']*data['Twb']
    df_op['FRwater_sq*Twb'] = (data['FRwater']**2)*data['Twb']
    df_op['Twb_sq'] = data['Twb']**2
    df_op['FRair*Twb_sq'] = data['FRair']*(data['Twb']**2)
    df_op['FRwater*Twb_sq'] = data['FRwater']*(data['Twb']**2)
    df_op['Twb_tri'] = data['Twb']**3
    df_op['Tr'] = Tr
    df_op['FRair*Tr'] = data['FRair']*Tr
    df_op['FRair_sq*Tr'] = (data['FRair']**2)*Tr
    df_op['FRwater*Tr'] = data['FRwater']*Tr
    df_op['FRair*FRwater*Tr'] = data['FRair']*data['FRwater']*Tr
    df_op['FRwater_sq*Tr'] = (data['FRwater']**2)*Tr
    df_op['Twb*Tr'] = data['Twb']*Tr
    df_op['FRair*Twb*Tr'] = data['FRair']*data['Twb']*Tr
    df_op['FRwater*Twb*Tr'] = data['FRwater']*data['Twb']*Tr
    df_op['Twb_sq*Tr'] = (data['Twb']**2)*Tr
    df_op['Tr_sq'] = Tr**2
    df_op['FRair*Tr_sq'] = data['FRair']*(Tr**2)
    df_op['FRwater*Tr_sq'] = data['FRwater']*(Tr**2)
    df_op['Twb*Tr_sq'] = data['Twb']*(Tr**2)
    df_op['Tr_tri'] = Tr**3
    y = approach.values
    x = df_op.values
    model = LinearRegression()
    model.fit(x,y)
    r_sq = model.score(x, y)
    print('coefficient of determination:', r_sq)
    result = {'intercept':model.intercept_}
    for i in range(df_op.shape[1]):
        result[df_op.columns[i]] = model.coef_[i]
    result['r_sq'] = r_sq
    return result    

In [206]:
def CT_YorkCalc(data):
    # model reference: E+ Engineering Reference Page 990
    approach = data['water_outlet']-data['Twb']
    Tr = data['water_inlet']-data['water_outlet']
    LGratio = data['FRwater']/data['FRair']
    df_op = pd.DataFrame()
    df_op['Twb'] = data['Twb']
    df_op['Twb_sq'] = data['Twb']**2
    df_op['Tr'] = Tr
    df_op['Twb*Tr'] = data['Twb']*Tr
    df_op['Twb_sq*Tr'] = (data['Twb']**2)*Tr
    df_op['Tr_sq'] = Tr**2
    df_op['Twb*Tr_sq'] = data['Twb']*(Tr**2)
    df_op['Twb_sq*Tr_sq'] = (data['Twb']**2)*(Tr**2)
    df_op['LGratio'] = LGratio
    df_op['Twb*LGratio'] = data['Twb']*LGratio
    df_op['Twb_sq*LGratio'] = (data['Twb']**2)*LGratio
    df_op['Tr*LGratio'] = Tr*LGratio
    df_op['Twb*Tr*LGratio'] = data['Twb']*Tr*LGratio
    df_op['Twb_sq*Tr*LGratio'] = (data['Twb']**2)*Tr*LGratio
    df_op['Tr_sq*LGratio'] = (Tr**2)*LGratio
    df_op['Twb*Tr_sq*LGratio'] = data['Twb']*(Tr**2)*LGratio
    df_op['Twb_sq*Tr_sq*LGratio'] = (data['Twb']**2)*(Tr**2)*LGratio
    df_op['LGratio_sq'] = LGratio**2
    df_op['Twb*LGratio_sq'] = data['Twb']*(LGratio**2)
    df_op['Twb_sq*LGratio_sq'] = (data['Twb']**2)*(LGratio**2)
    df_op['Tr*LGratio_sq'] = Tr*(LGratio**2)
    df_op['Twb*Tr*LGratio_sq'] = data['Twb']*Tr*(LGratio**2)
    df_op['Tr_sq*LGratio_sq'] = (Tr**2)*(LGratio**2)
    df_op['Twb*Tr_sq*LGratio_sq'] = data['Twb']*(Tr**2)*(LGratio**2)
    df_op['Twb_sq*Tr_sq*LGratio_sq'] = (data['Twb']**2)*(Tr**2)*(LGratio**2)

    y = approach.values
    x = df_op.values
    model = LinearRegression()
    model.fit(x,y)
    r_sq = model.score(x, y)
    print('coefficient of determination:', r_sq)
    result = {'intercept':model.intercept_}
    for i in range(df_op.shape[1]):
        result[df_op.columns[i]] = model.coef_[i]
    result['r_sq'] = r_sq
    return result    

## Regression and saving results

In [246]:
coefficient_df_coolTools = pd.DataFrame()
coefficient_df_YorkCalc = pd.DataFrame()
for i in range(1,10):
    data = prepare_data('CT'+str(i))
    coefficient_coolTools = CT_coolTools(data)
    coefficient_YorkCalc = CT_YorkCalc(data)
    df_coolTools = pd.DataFrame(coefficient_coolTools, index=[i])
    df_YorkCalc = pd.DataFrame(coefficient_YorkCalc, index=[i])
    coefficient_df_coolTools = pd.concat([coefficient_df_coolTools,df_coolTools])
    coefficient_df_YorkCalc = pd.concat([coefficient_df_YorkCalc,df_YorkCalc])

C:\Users\Walter\Anaconda3\lib\site-packages\dateutil\parser\_parser.py:1206: UnknownTimezoneWarning: tzname PDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)
C:\Users\Walter\Anaconda3\lib\site-packages\dateutil\parser\_parser.py:1206: UnknownTimezoneWarning: tzname PST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


filtering out 0.10580912863070539 data points according to the power_to_fanSpeed ratio criterion
coefficient of determination: 0.4405215080220876
coefficient of determination: 0.233547455107047
filtering out 0.10812320459623365 data points according to the power_to_fanSpeed ratio criterion
coefficient of determination: 0.4140993567631669
coefficient of determination: 0.20205109004927746
filtering out 0.1176793156393946 data points according to the power_to_fanSpeed ratio criterion
coefficient of determination: 0.5642093061195463
coefficient of determination: 0.3952249542113356
filtering out 0.10640166028097063 data points according to the power_to_fanSpeed ratio criterion
coefficient of determination: 0.424611061061224
coefficient of determination: 0.22008440993296674
filtering out 0.11022364217252396 data points according to the power_to_fanSpeed ratio criterion
coefficient of determination: 0.42083359633865103
coefficient of determination: 0.20362407606367583
filtering out 0.82083543

In [252]:
coefficient_df_coolTools.T.to_csv('coolingTower_coolTools.csv')
coefficient_df_YorkCalc.T.to_csv('coolingTower_YorkCalc.csv')